# Avances del Proyecto 1

## Importación y unificación de datos

In [1]:
import pandas as pd
import os

columnas = ['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 
            'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 
            'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

carpeta = 'data/csv/'
dfs = []

for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):
        ruta_completa = os.path.join(carpeta, archivo)
        temp_df = pd.read_csv(ruta_completa)
        print(f'Leyendo {archivo} con {temp_df.shape[0]} centros educativos')
        dfs.append(pd.read_csv(ruta_completa))

df = pd.concat(dfs, ignore_index=True)


Leyendo altaverapaz.csv con 294 centros educativos
Leyendo bajaverapaz.csv con 94 centros educativos
Leyendo chimaltenango.csv con 300 centros educativos
Leyendo chiquimula.csv con 136 centros educativos
Leyendo ciudadcapital.csv con 860 centros educativos
Leyendo elprogreso.csv con 97 centros educativos
Leyendo escuintla.csv con 393 centros educativos
Leyendo guatemala.csv con 1036 centros educativos
Leyendo huehuetenango.csv con 295 centros educativos
Leyendo izabal.csv con 273 centros educativos
Leyendo jalapa.csv con 121 centros educativos
Leyendo jutiapa.csv con 296 centros educativos
Leyendo peten.csv con 270 centros educativos
Leyendo quetzaltenango.csv con 365 centros educativos
Leyendo quiche.csv con 184 centros educativos
Leyendo retalhuleu.csv con 272 centros educativos
Leyendo sacatepequez.csv con 206 centros educativos
Leyendo sanmarcos.csv con 431 centros educativos
Leyendo santarosa.csv con 133 centros educativos
Leyendo solola.csv con 111 centros educativos
Leyendo such

### Obtener valores de columnas generales

In [5]:
for col in ['NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']:
    print(f'Datos únicos de la columna {col}:\n{df[col].unique()}')

Datos únicos de la columna NIVEL:
['DIVERSIFICADO']
Datos únicos de la columna SECTOR:
['PRIVADO' 'OFICIAL' 'MUNICIPAL' 'COOPERATIVA']
Datos únicos de la columna AREA:
['URBANA' 'RURAL' 'SIN ESPECIFICAR']
Datos únicos de la columna STATUS:
['ABIERTA']
Datos únicos de la columna MODALIDAD:
['MONOLINGUE' 'BILINGUE']
Datos únicos de la columna JORNADA:
['MATUTINA' 'VESPERTINA' 'DOBLE' 'NOCTURNA' 'SIN JORNADA' 'INTERMEDIA']
Datos únicos de la columna PLAN:
['DIARIO(REGULAR)' 'FIN DE SEMANA' 'A DISTANCIA' 'SEMIPRESENCIAL'
 'SEMIPRESENCIAL (FIN DE SEMANA)' 'SEMIPRESENCIAL (UN DÍA A LA SEMANA)'
 'VIRTUAL A DISTANCIA' 'SEMIPRESENCIAL (DOS DÍAS A LA SEMANA)' 'SABATINO'
 'INTERCALADO' 'DOMINICAL' 'MIXTO']
Datos únicos de la columna DEPARTAMENTAL:
['ALTA VERAPAZ' 'BAJA VERAPAZ' 'CHIMALTENANGO' 'CHIQUIMULA'
 'GUATEMALA NORTE' 'GUATEMALA ORIENTE' 'GUATEMALA OCCIDENTE'
 'GUATEMALA SUR' 'EL PROGRESO' 'ESCUINTLA' 'HUEHUETENANGO' 'IZABAL'
 'JALAPA' 'JUTIAPA' 'PETÉN' 'QUETZALTENANGO' 'QUICHÉ' 'QUICHÉ NO

In [4]:
df.shape

(6584, 17)

## **1. Descripción general del dataset**

A continuación, se describen los datos obtenidos del sitio web del [Mineduc](https://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/):

Los datos corresponden a los **6584** centros educativos a nivel nacional que imparten **educación diversificada**. Están organizados en **23 archivos CSV**, uno por cada **departamento** de Guatemala. Cada archivo contiene **17 columnas** con información relevante sobre los establecimientos:


* `CODIGO`: Identificador único del establecimiento, con el formato `XX-XX-XXXX-XX`.

  * **Primer bloque (`XX`)**: Código del departamento (ej. `18` para Izabal).
  * **Segundo bloque (`XX`)**: Código del municipio dentro del departamento.
  * **Tercer bloque (`XXXX`)**: Identificador interno del centro educativo.
  * **Cuarto bloque (`XX`)**: Es siempre `46`, posiblemente indica el nivel educativo (diversificado) o el sistema de codificación actual del Mineduc.

* `DISTRITO`: Código del distrito escolar al que pertenece el establecimiento. Tiene el formato `XX-XXX`, donde el primer número corresponde al departamento, y los últimos tres identifican las zonas operativas o educativas regionales dentro del departamento. **Ejemplo:** `18-007` y `18-008` son distritos distintos dentro de Izabal.

* `DEPARTAMENTO`: Nombre del departamento (ej. `IZABAL`).

* `MUNICIPIO`: Municipio donde se ubica el establecimiento (ej. `PUERTO BARRIOS`).

* `ESTABLECIMIENTO`: Nombre oficial del centro educativo. Puede incluir el tipo (`COLEGIO`, `ESCUELA`, `INSTITUTO`) y detalles como mixto, cristiano, etc.

* `DIRECCION`: Ubicación del centro.

* `TELEFONO`: Número telefónico del establecimiento, si está disponible.

* `SUPERVISOR`: Nombre del supervisor distrital asignado al establecimiento.

* `DIRECTOR`: Nombre del director o responsable del centro educativo.

* `NIVEL`: Máximo nivel educativo ofrecido. En este caso todos son `DIVERSIFICADO`.

* `SECTOR`: Define el sector del establecimiento, puede ser:
    - **PRIVADO**
    - **OFICIAL**
    - **MUNICIPAL**
    - **COOPERATIVA**

* `AREA`: Área geográfica según categorización del Mineduc: **URBANA**, **RURAL** o **SIN ESPECIFICAR**.

* `STATUS`: Estado de funcionamiento del centro educativo. En estos datos, el valor para todos es `ABIERTA`.

* `MODALIDAD`: Tipo de modalidad educativa: `MONOLINGUE` o `BILINGUE`.

* `JORNADA`: Jornada en que opera el centro: **MATUTINA**, **VESPERTINA**, **DOBLE**, **NOCTURNA**, **SIN JORNADA**, **INTERMEDIA**.

* `PLAN`: Plan educativo implementado:
    - **DIARIO(REGULAR)**
    - **FIN DE SEMANA**
    - **A DISTANCIA**
    - **SEMIPRESENCIAL**
    - **SEMIPRESENCIAL (FIN DE SEMANA)**
    - **SEMIPRESENCIAL (UN DÍA A LA SEMANA)**
    - **VIRTUAL A DISTANCIA**
    - **SEMIPRESENCIAL (DOS DÍAS A LA SEMANA)**
    - **SABATINO**
    - **INTERCALADO**
    - **DOMINICAL**
    - **MIXTO**

* `DEPARTAMENTAL`: Nombre del departamento al que pertenece el centro educativo. En algunos casos, describe subdivisiones regionales dentro de departamentos con alta densidad de establecimientos, como **GUATEMALA NORTE**, **GUATEMALA ORIENTE**, **GUATEMALA OCCIDENTE**, **GUATEMALA SUR**, o distinciones como **QUICHÉ** y **QUICHÉ NORTE**.

## 2. Variables que requieren limpieza

<!-- Enlistar las variables que tienen datos inconsistentes, errores comunes, o problemas de formato. Comentar brevemente por qué requieren limpieza. -->

Ejemplo:

* `ESTABLECIMIENTO`: hay diferencias por uso de mayúsculas, comillas, acentos inconsistentes.
* `DIRECCION`: muchas variaciones, errores ortográficos o campos incompletos.
* `SUPERVISOR` y `DIRECTOR`: hay diferencias por uso de mayúsculas/minúsculas o nombres mal escritos.
* `TELEFONO`: hay algunos vacíos, valores con caracteres no numéricos.

In [10]:
# Examinar una muestra de datos para identificar columnas que necesitan limpieza
print("Registros para analizar:")
df.sample(3)

Registros para analizar:


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
5318,03-01-0210-46,03-002,SACATEPEQUEZ,ANTIGUA GUATEMALA,COLEGIO ANTIGUA,CALLE DEL HERMANO PEDRO FINAL PRIMAVERA ENTRAD...,78326275,ZOILA ESTHELA JONFE OROZCO,JOSÉ SALOMÓN RODRÍGUEZ ROGEL,DIVERSIFICADO,PRIVADO,RURAL,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),SACATEPÉQUEZ
4820,09-23-0036-46,09-020,QUETZALTENANGO,LA ESPERANZA,COLEGIO HIGHLAND QUETZALTENANGO,1A. CALLE 5-64 ZONA 1,54827205,JORGE MARIO ROJAS FERNANDEZ,MILNA LISBETH VILLATORO BARRIOS,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
3338,13-05-0055-46,13-014,HUEHUETENANGO,NENTON,"COLEGIO INTERCULTURAL PRIVADO MIXTO ""CIENCIA A...",CABECERA MUNICIPAL,57381713,JOSELINO SAMAYOA CASTILLO,MIGUEL ANGEL HERNÁNDEZ TÓRREZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (FIN DE SEMANA),HUEHUETENANGO


In [12]:
# Analizar Columnas
columns_to_check = ['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR']

for col in columns_to_check:
    print(f"\n==== Análisis de la columna {col} ====")
    # valores únicos
    print(f"Cantidad de valores únicos: {df[col].nunique()}")
    print(f"Ejemplo/Muestra de valores: {df[col].sample(3).tolist()}")
    # valores nulos
    null_count = df[col].isna().sum()
    print(f"Valores nulos: {null_count} ({null_count/len(df)*100:.2f}%)")
    # valores vacíos
    empty_count = (df[col] == '').sum()
    print(f"Valores vacíos: {empty_count} ({empty_count/len(df)*100:.2f}%)")
    if col == 'TELEFONO':
        # Verificación sii hay valores no numéricos
        non_numeric = df[col].str.replace(r'[0-9\s\-\(\)\+]', '', regex=True).str.len() > 0
        print(f"Valores con caracteres no numéricos: {non_numeric.sum()} ({non_numeric.sum()/len(df)*100:.2f}%)")
    
    if col == 'ESTABLECIMIENTO':
        # diferencias por uso de mayúsculas/minúsculas
        lowercase_count = df[col].str.islower().sum()
        uppercase_count = df[col].str.isupper().sum()
        mixed_count = len(df) - lowercase_count - uppercase_count
        print(f"Valores en minúsculas: {lowercase_count} ({lowercase_count/len(df)*100:.2f}%)")
        print(f"Valores en mayúsculas: {uppercase_count} ({uppercase_count/len(df)*100:.2f}%)")
        print(f"Valores mixtos: {mixed_count} ({mixed_count/len(df)*100:.2f}%)")


==== Análisis de la columna ESTABLECIMIENTO ====
Cantidad de valores únicos: 3989
Ejemplo/Muestra de valores: ['COLEGIO EVANGÉLICO MIXTO "LA VID VERDADERA"', 'INSTITUTO NORMAL MIXTO JUAN DE LEON', "CENTRO ESCOLAR 'EL ROBLE'"]
Valores nulos: 0 (0.00%)
Valores vacíos: 0 (0.00%)
Valores en minúsculas: 0 (0.00%)
Valores en mayúsculas: 6584 (100.00%)
Valores mixtos: 0 (0.00%)

==== Análisis de la columna DIRECCION ====
Cantidad de valores únicos: 4490
Ejemplo/Muestra de valores: ['INTERIOR INEB FRANCISCO MARROQUÍN, AVENIDA VICENTE COZZA', '6TA. AVE.  ENTRE 3RA. Y 4TA. CALLE  ZONA 1 SANTA ELENA', 'COLONIA VILLA DEL ROSARIO, BANANERA']
Valores nulos: 2 (0.03%)
Valores vacíos: 0 (0.00%)

==== Análisis de la columna TELEFONO ====
Cantidad de valores únicos: 4208
Ejemplo/Muestra de valores: [77757007, 79422150.0, 45169723.0]
Valores nulos: 45 (0.68%)
Valores vacíos: 0 (0.00%)
Valores con caracteres no numéricos: 0 (0.00%)

==== Análisis de la columna SUPERVISOR ====
Cantidad de valores únicos: 

1. **ESTABLECIMIENTO**: 
   - Contiene inconsistencias en el uso de acentos (por ejemplo, "EDUCACION" vs "EDUCACIÓN", "TECNICO" vs "TÉCNICO").
   - El 23.88% de los registros contienen comillas (").
   - Existen 1,185 establecimientos con nombres duplicados pero con códigos diferentes, lo que indica posibles variantes del mismo centro educativo con diferentes jornadas o planes.

2. **DIRECCION**:
   - Presenta 2 valores nulos (0.03%).
   - Contiene 47 direcciones con menos de 10 caracteres (0.71%), sugiriendo información incompleta.
   - Alta variabilidad en el formato y estilo de escritura.

3. **TELEFONO**:
   - Tiene 45 valores nulos (0.68%).
   - El formato no es consistente, aunque el 73.80% son números de 8 dígitos sin separadores.
   - Algunos están almacenados como números y otros como texto.

4. **SUPERVISOR** y **DIRECTOR**:
   - Para DIRECTOR hay 23 valores nulos (0.35%).
   - Posibles inconsistencias en el formato de nombres (espacios dobles, uso de mayúsculas/minúsculas, acentos).

## 3. Estrategia de limpieza

<!-- Proponer pasos concretos para limpiar los datos. Puede variar por columna, indicando operaciones como normalización, eliminación de duplicados, corrección de errores, etc. -->

### `ESTABLECIMIENTO`

1. Convertir todos los nombres a mayúsculas.
2. Eliminar espacios duplicados y caracteres especiales innecesarios.
3. Corregir errores de escritura comunes (ej. "COLEGO" → "COLEGIO").
4. Eliminar registros duplicados si el nombre y la dirección coinciden.

In [13]:
# verificar duplicados y analizar el formato del CODIGO
print("Verificación de duplicados:")
duplicate_count = df.duplicated().sum()
print(f"Filas completamente duplicadas: {duplicate_count}")

# verificar establecimientos potencialmente duplicados (mismo nombre, diferente código)
est_counts = df['ESTABLECIMIENTO'].value_counts()
duplicated_est = est_counts[est_counts > 1]
print(f"\nEstablecimientos con nombres duplicados: {len(duplicated_est)}")
print("Algunos establecimientos duplicados:")
for est in duplicated_est.head(3).index:
    print(f"\n{est}:")
    display(df[df['ESTABLECIMIENTO'] == est][['CODIGO', 'MUNICIPIO', 'JORNADA', 'PLAN']].head(3))

# Verificar formato del CODIGO
print("\nAnálisis del formato de CODIGO:")
valid_format = df['CODIGO'].str.match(r'^\d{2}-\d{2}-\d{4}-\d{2}$')
print(f"Códigos con formato válido (XX-XX-XXXX-XX): {valid_format.sum()} ({valid_format.sum()/len(df)*100:.2f}%)")
print(f"Códigos con formato inválido: {(~valid_format).sum()} ({(~valid_format).sum()/len(df)*100:.2f}%)")

Verificación de duplicados:
Filas completamente duplicadas: 0

Establecimientos con nombres duplicados: 1185
Algunos establecimientos duplicados:

INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA:


,CODIGO,MUNICIPIO,JORNADA,PLAN
7,16-01-0428-46,COBAN,VESPERTINA,DIARIO(REGULAR)
24,16-01-0616-46,COBAN,VESPERTINA,DIARIO(REGULAR)
25,16-01-0642-46,COBAN,VESPERTINA,DIARIO(REGULAR)



INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA:


,CODIGO,MUNICIPIO,JORNADA,PLAN
49,16-01-0926-46,COBAN,NOCTURNA,DIARIO(REGULAR)
50,16-01-0927-46,COBAN,NOCTURNA,DIARIO(REGULAR)
133,16-06-0090-46,SAN MIGUEL TUCURU,VESPERTINA,DIARIO(REGULAR)



CENTRO DE EDUCACIÓN EXTRAESCOLAR -CEEX-:


,CODIGO,MUNICIPIO,JORNADA,PLAN
55,16-01-0994-46,COBAN,DOBLE,DIARIO(REGULAR)
97,16-03-0158-46,SAN CRISTOBAL VERAPAZ,DOBLE,SEMIPRESENCIAL
105,16-03-0266-46,SAN CRISTOBAL VERAPAZ,DOBLE,DIARIO(REGULAR)



Análisis del formato de CODIGO:
Códigos con formato válido (XX-XX-XXXX-XX): 6584 (100.00%)
Códigos con formato inválido: 0 (0.00%)


In [14]:
#inconsistencias en ortografía y capitalización
import re

# ESTABLECIMIENTO para detectar problemas comunes
print("Análisis de problemas comunes en ESTABLECIMIENTO:")

# Verificar uso de comillas
comillas_count = df['ESTABLECIMIENTO'].str.contains('"').sum()
print(f"Nombres con comillas (\"):  {comillas_count} ({comillas_count/len(df)*100:.2f}%)")

# Verificar uso inconsistente de acentos
print("\nPosibles inconsistencias en acentos:")
accent_variations = [
    ['EDUCACION', 'EDUCACIÓN'],
    ['BASICO', 'BÁSICO'],
    ['TECNICO', 'TÉCNICO'],
    ['INFORMATICA', 'INFORMÁTICA']
]

for variations in accent_variations:
    counts = [df['ESTABLECIMIENTO'].str.contains(var).sum() for var in variations]
    print(f"{variations[0]}: {counts[0]} vs {variations[1]}: {counts[1]}")

# TELEFONO para verificar formato
print("\nFormato de TELEFONO:")
# Verificar formatos comunes (8 dígitos, con guiones, etc.)
tel_8_digits = df['TELEFONO'].astype(str).str.match(r'^\d{8}$')
print(f"Teléfonos con formato de 8 dígitos sin separadores: {tel_8_digits.sum()} ({tel_8_digits.sum()/len(df)*100:.2f}%)")

# Verificar DIRECCION
print("\nAnálisis de DIRECCION:")
# Verificar registros con direcciones muy cortas (posible falta de información)
short_dir = df['DIRECCION'].astype(str).str.len() < 10
print(f"Direcciones con menos de 10 caracteres: {short_dir.sum()} ({short_dir.sum()/len(df)*100:.2f}%)")

Análisis de problemas comunes en ESTABLECIMIENTO:
Nombres con comillas ("):  1572 (23.88%)

Posibles inconsistencias en acentos:
EDUCACION: 599 vs EDUCACIÓN: 494
BASICO: 6 vs BÁSICO: 1
TECNICO: 173 vs TÉCNICO: 145
INFORMATICA: 71 vs INFORMÁTICA: 48

Formato de TELEFONO:
Teléfonos con formato de 8 dígitos sin separadores: 4859 (73.80%)

Análisis de DIRECCION:
Direcciones con menos de 10 caracteres: 47 (0.71%)


## 4. Consideraciones adicionales

Basado en el análisis realizado, estas son las consideraciones adicionales que se deben tener en cuenta durante el proceso de limpieza y análisis de los datos:

### Consideraciones sobre identificación de establecimientos

* **Verificación de unicidad de CODIGO**: El análisis confirma que todos los códigos tienen el formato válido (`XX-XX-XXXX-XX`) y son únicos. Este campo debe mantenerse como identificador primario de cada registro.

* **Detección de centros educativos duplicados**: Muchos establecimientos aparecen con el mismo nombre pero distintos códigos. Esto se debe principalmente a que ofrecen diferentes jornadas (MATUTINA, VESPERTINA, NOCTURNA) o planes educativos (DIARIO, FIN DE SEMANA). No son verdaderos duplicados, sino diferentes modalidades del mismo centro físico.

* **Creación de identificador de establecimiento físico**: Se recomienda generar un nuevo identificador que agrupe los diferentes registros que corresponden al mismo centro educativo físico. Este se puede crear combinando el nombre normalizado y la dirección.

### Consideraciones sobre integridad de relaciones

* **Verificación de relaciones geográficas**: Es importante validar la consistencia entre DEPARTAMENTO, MUNICIPIO y DISTRITO. Cada distrito debería pertenecer a un único departamento, y cada municipio a un único departamento.

* **Análisis de ubicaciones repetidas**: Existen direcciones que aparecen en múltiples registros, lo que indica que varios centros educativos pueden estar operando en la misma ubicación física, posiblemente compartiendo instalaciones pero con administraciones diferentes.

### Consideraciones sobre valores ausentes o incompletos

* **Datos de contacto incompletos**: Para los registros con TELEFONO o DIRECTOR ausentes, se debe decidir si se completan con valores como "NO DISPONIBLE" o si se tratan con técnicas más avanzadas.

* **Direcciones incompletas**: Las direcciones muy cortas o genéricas deberían revisarse manualmente o complementarse con información adicional si está disponible.

### Consideraciones sobre enriquecimiento de datos

* **Posibles variables derivadas**:
  - Crear una columna para identificar el **tipo de establecimiento** (COLEGIO, INSTITUTO, ESCUELA) extrayéndolo del nombre.
  - Añadir una columna de **región geográfica** que agrupe departamentos por zonas (norte, sur, oriente, occidente, central).
  - Generar una variable de **clasificación** que combine SECTOR, MODALIDAD y PLAN para facilitar análisis comparativos.

* **Geocodificación**: Evaluar la posibilidad de convertir las direcciones en coordenadas geográficas para análisis espaciales y visualización en mapas.

### Consideraciones sobre técnicas de normalización avanzadas

* **Detección de similitud de nombres**: Usar técnicas de coincidencia difusa (fuzzy matching) o distancia de edición para identificar nombres de establecimientos que podrían ser el mismo pero están escritos con variaciones menores.

* **Normalización lingüística**: Estandarizar términos específicos del ámbito educativo que pueden aparecer con diferentes grafías o abreviaturas.

> **Nota importante**: Como se observó en el ejemplo de Quiché con códigos **14-14-0070-46** y **14-14-0083-46**, donde uno opera en jornada **DOBLE** con plan **FIN DE SEMANA** y el otro en jornada **VESPERTINA** con plan **DIARIO(REGULAR)**, estos registros representan diferentes ofertas educativas del mismo centro y no deben eliminarse como duplicados.

In [16]:
# 1. Misma dirección pero diferentes nombres
direcciones_counts = df.groupby('DIRECCION').size()
direcciones_duplicadas = direcciones_counts[direcciones_counts > 1].sort_values(ascending=False)

print(f"Existen {len(direcciones_duplicadas)} direcciones que aparecen en más de un registro.")
print(f"Las 5 direcciones más repetidas son:")
for direccion in direcciones_duplicadas.head(5).index:
    print(f"\nDirección: '{direccion}'")
    print(df[df['DIRECCION'] == direccion][['CODIGO', 'ESTABLECIMIENTO', 'JORNADA', 'PLAN']].head(3))

# 2. Agrupamiento establecimientos similares
def simplificar_nombre(nombre):
    # Convertir a mayúsculas y eliminar comillas
    nombre = nombre.upper().replace('"', '')
    # Eliminar palabras genéricas que pueden causar variaciones
    palabras_eliminar = ['PRIVADO', 'MIXTO', 'COLEGIO', 'INSTITUTO', 'ESCUELA', 'CENTRO', 'DE', 'Y', 'EDUCACION']
    for palabra in palabras_eliminar:
        nombre = nombre.replace(f" {palabra} ", " ")
    # Eliminar espacios múltiples
    return " ".join(nombre.split())

# columna temporal con nombres simplificados
df['NOMBRE_SIMPLIFICADO'] = df['ESTABLECIMIENTO'].apply(simplificar_nombre)

# ocurrencias de nombres simplificados
nombres_simples_counts = df['NOMBRE_SIMPLIFICADO'].value_counts()
nombres_simples_duplicados = nombres_simples_counts[nombres_simples_counts > 1].sort_values(ascending=False)

print(f"\nDespués de simplificar nombres, existen {len(nombres_simples_duplicados)} nombres potencialmente similares.")
print(f"Los 5 nombres más comunes después de simplificación son:")
for nombre in nombres_simples_duplicados.head(5).index:
    print(f"\nNombre simplificado: '{nombre}'")
    print(f"Nombres originales:")
    print(df[df['NOMBRE_SIMPLIFICADO'] == nombre]['ESTABLECIMIENTO'].sample(min(3, nombres_simples_counts[nombre])).tolist())

# 3. Integridad de relaciones entre DEPARTAMENTO, MUNICIPIO y DISTRITO
print("\nVerificando integridad de relaciones geográficas:")
municipios_por_depto = df.groupby(['DEPARTAMENTO', 'MUNICIPIO']).size().reset_index()
print(f"Hay {len(municipios_por_depto)} combinaciones únicas de DEPARTAMENTO-MUNICIPIO")

# relación entre distrito y departamento
distrito_depto = df[['DISTRITO', 'DEPARTAMENTO']].drop_duplicates()
print(f"Hay {len(distrito_depto)} combinaciones únicas de DISTRITO-DEPARTAMENTO")

# distritos que pertenecen a más de un departamento (posible inconsistencia)
distrito_counts = distrito_depto.groupby('DISTRITO').size()
distritos_multiples = distrito_counts[distrito_counts > 1]
if len(distritos_multiples) > 0:
    print(f"Hay {len(distritos_multiples)} distritos asignados a más de un departamento (posible inconsistencia)")
    for distrito in distritos_multiples.index:
        print(f"\nDistrito: {distrito}")
        print(df[df['DISTRITO'] == distrito][['DEPARTAMENTO']].drop_duplicates().values.flatten())

Existen 1039 direcciones que aparecen en más de un registro.
Las 5 direcciones más repetidas son:

Dirección: 'CABECERA MUNICIPAL'
             CODIGO                                ESTABLECIMIENTO  \
361   15-05-0014-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   
367   15-06-0006-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   
1737  02-04-0004-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA   

         JORNADA             PLAN  
361   VESPERTINA  DIARIO(REGULAR)  
367   VESPERTINA  DIARIO(REGULAR)  
1737  VESPERTINA  DIARIO(REGULAR)  

Dirección: 'BARRIO EL CENTRO'
            CODIGO                                ESTABLECIMIENTO     JORNADA  \
133  16-06-0090-46  INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA  VESPERTINA   
162  16-08-0208-46  INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA  VESPERTINA   
241  16-13-0254-46  INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA    MATUTINA   

                PLAN  
133  DIARIO(REGULAR)  
162  DIARIO(REGULAR)  
241  DIARIO(REGULAR

### CSV TEMPORALES DE PRUEBA

In [22]:
# datos limpios
import os
import shutil

carpeta_destino = 'data_temporal1'
carpeta_destino_csv = os.path.join(carpeta_destino, 'csv')

# Crear la carpeta si no existe
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
    print(f"Carpeta '{carpeta_destino}' creada.")

if not os.path.exists(carpeta_destino_csv):
    os.makedirs(carpeta_destino_csv)
    print(f"Carpeta '{carpeta_destino_csv}' creada.")

Carpeta 'data_temporal1' creada.
Carpeta 'data_temporal1\csv' creada.


In [23]:
# Implementar las transformaciones de limpieza
# Crear una copia del DataFrame original para no modificarlo
df_limpio = df.copy()

print("Aplicando transformaciones de limpieza...")

# 1. Limpieza de ESTABLECIMIENTO
print("Limpiando columna ESTABLECIMIENTO...")

# normalizar acentos en palabras comunes
def normalizar_acentos(texto):
    reemplazos = {
        'EDUCACION': 'EDUCACIÓN',
        'BASICO': 'BÁSICO',
        'TECNICO': 'TÉCNICO',
        'INFORMATICA': 'INFORMÁTICA',
        'TECNOLOGICO': 'TECNOLÓGICO',
        'CIENTIFICO': 'CIENTÍFICO',
        'MECANICA': 'MECÁNICA',
        'ELECTRONICA': 'ELECTRÓNICA',
        'GRAFICA': 'GRÁFICA',
        'DISENIO': 'DISEÑO',
        'COMERCIO': 'COMERCIO',
        'ADMINISTRACION': 'ADMINISTRACIÓN',
        'MUSICA': 'MÚSICA',
        'ECONOMIA': 'ECONOMÍA',
        'PEDAGOGICO': 'PEDAGÓGICO',
        'TECNOLOGIA': 'TECNOLOGÍA',
        'ELECTROMECANICA': 'ELECTROMECÁNICA',
        'AGRICOLA': 'AGRÍCOLA',
        'MECANICO': 'MECÁNICO',
        'QUIMICA': 'QUÍMICA',
        'ELECTRONICO': 'ELECTRÓNICO'
    }
    
    for palabra, reemplazo in reemplazos.items():
        texto = re.sub(r'\b' + palabra + r'\b', reemplazo, texto)
    
    return texto

# aplicar normalización de acentos
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].apply(normalizar_acentos)

# eliminación de  comillas y caracteres especiales innecesarios
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace('"', '')
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace('\'', '')

#  eliminación de espacios duplicados
df_limpio['ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].str.replace(r'\s+', ' ', regex=True).str.strip()

# 2. Limpieza de DIRECCION
print("Limpiando columna DIRECCION...")

# Completar valores nulos
df_limpio['DIRECCION'] = df_limpio['DIRECCION'].fillna('SIN DIRECCIÓN')

# Normalizar abreviaturas comunes
direccion_reemplazos = {
    r'\bAV\b': 'AVENIDA',
    r'\bC\b': 'CALLE',
    r'\bCALLE\s+(\d+)': r'CALLE \1',
    r'\bAVENIDA\s+(\d+)': r'AVENIDA \1',
    r'\bZONA\s+(\d+)': r'ZONA \1',
    r'\bZ\.\s*(\d+)': r'ZONA \1',
    r'\bZ\s+(\d+)': r'ZONA \1',
    r'\bLOT\b': 'LOTE',
    r'\bSECTOR\s+(\d+)': r'SECTOR \1',
    r'\bSECT\s+(\d+)': r'SECTOR \1',
    r'\bKM\b': 'KILÓMETRO',
    r'\bKM\.\s*(\d+)': r'KILÓMETRO \1',
}

for patron, reemplazo in direccion_reemplazos.items():
    df_limpio['DIRECCION'] = df_limpio['DIRECCION'].str.replace(patron, reemplazo, regex=True)

# 3. limpieza de TELEFONO
print("Limpiando columna TELEFONO...")

df_limpio['TELEFONO'] = df_limpio['TELEFONO'].astype(str)

# Función para estandarizar el formato de teléfono
def estandarizar_telefono(telefono):
    if pd.isna(telefono) or telefono == 'nan':
        return 'NO DISPONIBLE'
    
    # Eliminar caracteres no numéricos
    solo_numeros = re.sub(r'[^\d]', '', telefono)
    # Si después de limpieza queda vacío
    if not solo_numeros:
        return 'NO DISPONIBLE'
    
    # Si tiene 8 dígitos, es un formato válido
    if len(solo_numeros) == 8:
        return solo_numeros
    
    # Si tiene más o menos dígitos, puede ser un error
    if len(solo_numeros) < 8:
        return 'FORMATO INCORRECTO: ' + telefono
    
    # Si tiene más de 8 dígitos, tomar los últimos 8
    return solo_numeros[-8:]

df_limpio['TELEFONO'] = df_limpio['TELEFONO'].apply(estandarizar_telefono)

# 4. SUPERVISOR y DIRECTOR
print("Limpiando columnas SUPERVISOR y DIRECTOR...")

# Función para normalizar nombres
def normalizar_nombre(nombre):
    if pd.isna(nombre):
        return 'SIN ASIGNAR'
    
    # Convertir a mayúsculas
    nombre = nombre.upper()
    
    # Eliminar espacios duplicados
    nombre = re.sub(r'\s+', ' ', nombre).strip()
    
    return nombre

df_limpio['SUPERVISOR'] = df_limpio['SUPERVISOR'].apply(normalizar_nombre)
df_limpio['DIRECTOR'] = df_limpio['DIRECTOR'].apply(normalizar_nombre)

# 5. Creación de  columnas derivadas
print("Creando columnas derivadas...")

# Extraer el tipo de establecimiento del nombre
def extraer_tipo_establecimiento(nombre):
    tipos = ['COLEGIO', 'INSTITUTO', 'ESCUELA', 'LICEO', 'CENTRO', 'ACADEMIA']
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return 'OTRO'

df_limpio['TIPO_ESTABLECIMIENTO'] = df_limpio['ESTABLECIMIENTO'].apply(extraer_tipo_establecimiento)

# Crear un identificador único de establecimiento físico (combinando nombre simplificado y dirección)
df_limpio['ID_ESTABLECIMIENTO_FISICO'] = df_limpio['NOMBRE_SIMPLIFICADO'] + '_' + df_limpio['DIRECCION'].str.replace(' ', '_')

# Agrupar departamentos por región
def asignar_region(departamento):
    regiones = {
        'NORTE': ['ALTA VERAPAZ', 'BAJA VERAPAZ', 'PETÉN'],
        'SUR': ['ESCUINTLA', 'SANTA ROSA', 'SUCHITEPÉQUEZ', 'RETALHULEU'],
        'ORIENTE': ['CHIQUIMULA', 'EL PROGRESO', 'IZABAL', 'JALAPA', 'JUTIAPA', 'ZACAPA'],
        'OCCIDENTE': ['HUEHUETENANGO', 'QUETZALTENANGO', 'QUICHÉ', 'SAN MARCOS', 'SOLOLÁ', 'TOTONICAPÁN'],
        'CENTRAL': ['CHIMALTENANGO', 'GUATEMALA', 'SACATEPÉQUEZ']
    }
    
    # Manejar subdivisiones de Guatemala
    if 'GUATEMALA' in departamento:
        return 'CENTRAL'
    
    # Manejar subdivisiones de Quiché
    if 'QUICHÉ' in departamento:
        return 'OCCIDENTE'
    
    for region, deptos in regiones.items():
        if departamento in deptos:
            return region
    
    return 'NO DEFINIDA'

df_limpio['REGION'] = df_limpio['DEPARTAMENTAL'].apply(asignar_region)

# Crear una clasificación combinada
df_limpio['CLASIFICACION'] = df_limpio['SECTOR'] + '_' + df_limpio['MODALIDAD'] + '_' + df_limpio['PLAN'].str.replace(' ', '_')

print(f"Limpieza completada. DataFrame resultante tiene {df_limpio.shape[0]} filas y {df_limpio.shape[1]} columnas.")

Aplicando transformaciones de limpieza...
Limpiando columna ESTABLECIMIENTO...
Limpiando columna DIRECCION...
Limpiando columna DIRECCION...
Limpiando columna TELEFONO...
Limpiando columnas SUPERVISOR y DIRECTOR...
Creando columnas derivadas...
Limpieza completada. DataFrame resultante tiene 6584 filas y 22 columnas.
Limpiando columna TELEFONO...
Limpiando columnas SUPERVISOR y DIRECTOR...
Creando columnas derivadas...
Limpieza completada. DataFrame resultante tiene 6584 filas y 22 columnas.


In [24]:
# Guardar los datos limpios en archivos CSV por departamento
print("Guardando datos limpios en archivos CSV...")
departamentos = df_limpio['DEPARTAMENTAL'].unique()

# archivo general con todos los datos
archivo_general = os.path.join(carpeta_destino, 'centros_educativos_completo.csv')
df_limpio.to_csv(archivo_general, index=False)
print(f"Archivo general guardado en: {archivo_general}")

# archivos individuales por cada uno de los departamentos
for departamento in departamentos:
    # archivo (sin caracteres especiales)
    nombre_archivo = departamento.lower().replace(' ', '_').replace('é', 'e')
    nombre_archivo = re.sub(r'[^\w\-_\.]', '', nombre_archivo)
    # Filtrar datos por departamento
    df_depto = df_limpio[df_limpio['DEPARTAMENTAL'] == departamento]

    ruta_archivo = os.path.join(carpeta_destino_csv, f"{nombre_archivo}.csv")
    # Guardar archivo
    df_depto.to_csv(ruta_archivo, index=False)
    
    print(f"Guardado archivo para {departamento} con {df_depto.shape[0]} registros: {ruta_archivo}")

print(f"\nProceso completado. {len(departamentos)} archivos CSV generados en la carpeta {carpeta_destino_csv}")

Guardando datos limpios en archivos CSV...
Archivo general guardado en: data_temporal1\centros_educativos_completo.csv
Guardado archivo para ALTA VERAPAZ con 294 registros: data_temporal1\csv\alta_verapaz.csv
Guardado archivo para BAJA VERAPAZ con 94 registros: data_temporal1\csv\baja_verapaz.csv
Guardado archivo para CHIMALTENANGO con 300 registros: data_temporal1\csv\chimaltenango.csv
Guardado archivo para CHIQUIMULA con 136 registros: data_temporal1\csv\chiquimula.csv
Guardado archivo para GUATEMALA NORTE con 536 registros: data_temporal1\csv\guatemala_norte.csv
Guardado archivo para GUATEMALA ORIENTE con 281 registros: data_temporal1\csv\guatemala_oriente.csv
Guardado archivo para GUATEMALA OCCIDENTE con 551 registros: data_temporal1\csv\guatemala_occidente.csv
Guardado archivo para GUATEMALA SUR con 528 registros: data_temporal1\csv\guatemala_sur.csv
Guardado archivo para EL PROGRESO con 97 registros: data_temporal1\csv\el_progreso.csv
Guardado archivo para ESCUINTLA con 393 regis